In [9]:
# #install python 3.9
# !sudo apt-get update -y
# !sudo apt-get install python3.9

# #change alternatives
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9.13

# #check python version
# !python --version
# #3.9.16


In [10]:
import sys
print(sys.version)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [11]:
CUDA_LAUNCH_BLOCKING=1

In [12]:
! pip install transformers

In [13]:
# !conda install pytorch torchvision -c pytorch

In [14]:
import pandas as pd
import numpy as np
import torch
import typing
import transformers
from transformers import TFAutoModel, AutoTokenizer, BertTokenizer, AutoConfig, BertModel
#%load_ext blackcellmagic

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My Drive/FintechDS
import os
os.listdir('/kaggle/input/')

['amazon-fake-reviews-scrapped']

In [16]:
df = pd.read_csv(
    "../input/amazon-fake-reviews-scrapped/df.csv",
    dtype={
        "product_title": str,
        "review_title": str,
        "review_text": str,
        "product_id": str,
    },
)
reviews_scraped = pd.read_csv(
    "../input/amazon-fake-reviews-scrapped/reviews_scraped.csv",
    dtype={"review_title": str, "review_text": str, "product_id": str},
)
print(df.shape)
print(reviews_scraped.shape)


#df = df[0:10000]

#reviews_scraped = reviews_scraped[0:20000]
reviews_scraped.fillna(0,inplace= True)
df.fillna(0,inplace=True)


(18107, 48)
(99889, 16)


In [17]:
df.columns

Index(['Unnamed: 0', 'rating', 'verified', 'PRODUCT_CATEGORY', 'product_id',
       'review_title', 'review_text', 'cat_0', 'cat_1', 'cat_2', 'cat_3',
       'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10',
       'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17',
       'cat_18', 'cat_19', 'cat_20', 'cat_21', 'cat_22', 'cat_23', 'cat_24',
       'cat_25', 'cat_26', 'cat_27', 'cat_28', 'cat_29', 'label',
       'text_sentiment', 'text_subjectivity', 'rating_count', 'rating_avg',
       'rating1', 'rating2', 'rating3', 'rating4', 'rating5', 'product_title'],
      dtype='object')

In [18]:
reviews_scraped.columns 

Index(['Unnamed: 0', 'product_id', 'review_title', 'rating_count',
       'rating_avg', 'rating1', 'rating2', 'rating3', 'rating4', 'rating5',
       'helpful', 'verified', 'review_text', 'rating', 'text_sentiment',
       'text_subjectivity'],
      dtype='object')

### Constants

In [19]:
BERT_TOKENIZER_LENGTH: int = 256
BERT_EMBEDDING_SIZE : int = 768
DEVICE :torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BERT_TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")
BERT_CONFIG, _ = AutoConfig.from_pretrained('bert-base-uncased', output_attention=True, return_unused_kwargs=True)
SCRAPED_REVIEW_LIMIT = 10
BATCH_SIZE=256

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Dataset


In [20]:
if sys.version_info >= (3, 8):
    from typing import TypedDict
else:
    from typing_extensions import TypedDict
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass


class BertInput(TypedDict):
    attention_mask: torch.Tensor
    """shape: (1,MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""
    input_ids: torch.Tensor
    """shape: (1,MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""


class BertInputBatch:
    def __init__(self, attention_mask: torch.Tensor, input_ids: torch.Tensor) -> None:
        self.attention_mask = attention_mask
        self.input_ids = input_ids

    @staticmethod
    def from_batch_encoding(
        batch_encoding: transformers.tokenization_utils_base.BatchEncoding,
    ):
        return BertInputBatch(
            torch.tensor(batch_encoding["attention_mask"]).to(DEVICE),
            torch.tensor(batch_encoding["input_ids"]).to(DEVICE),
        )

    attention_mask: torch.Tensor
    input_ids: torch.Tensor

    @property
    def shape(self) -> torch.Size:
        return self.input_ids.shape

    def __getitem__(self, i: int) -> BertInput:
        return {
            "attention_mask": self.attention_mask[i : i + 1, :],
            "input_ids": self.input_ids[i : i + 1, :],
        }


# class ReviewDataSetItemInput(TypedDict):
#     """one fake/real review with X context reviews from the same product"""

#     product_title_bert_input: BertInput
#     review_title_bert_input: BertInput
#     review_text_bert_input: BertInput

#     review_features: torch.Tensor
#     """shape: (4)    Rating [1-5; 1] + Verified Purchase [0-1; 1] + SA Valence + Subjectivity [0.0-1.0; 2]"""

#     product_features: torch.Tensor
#     """shape: (37)    Ratings (1-5) Ratio [0.0-1.0]; 5] + Rating Avg [f64; 1] + Rating Count [u64; 1] + Category [u64; 30]"""

#     scraped_review_features: torch.Tensor
#     """shape: (X, 4)    Rating [1-5; 1] + Verified Purchase [0-1; 1] + SA Valence + Subjectivity [0.0-1.0; 2]"""

#     scraped_helpful: torch.Tensor
#     """shape: (X, 1)    Helpfulness [u64; 1]"""

#     scraped_review_texts: BertInputBatch

#     scraped_review_titles: BertInputBatch

PACKED_DATASET_ROW_SIZE = 37 + 4 + 6 * BERT_TOKENIZER_LENGTH + SCRAPED_REVIEW_LIMIT * (1 + 4 + 4 * BERT_TOKENIZER_LENGTH)

class ReviewsDataSet(Dataset):
    """
    Args:
        df: pd.DataFrame of size M with columns:
            - rating (1-5)
            - verified (0-1)
            - product_id (str)
            - review_title (str)
            - review_text (str)
            - cat_0 - cat_29 (0-1)
            - label (0-1)
            - text_sentiment (0.0-1.0)
            - text_subjectivity (0.0-1.0)
            - rating_count (u64)
            - rating_avg (1.0-5.0)
            - rating1 - rating5  (0.0-1.0)
            - product_title (str)

        reviews_scraped:  pd.DataFrame of size N with columns:
            - product_id (str)
            - review_title (str)
            - review_text (str)
            - helpful (u64)
            - verified (0-1)
            - rating (1-5)
            - text_sentiment (0.0-1.0)
            - text_subjectivity (0.0-1.0)
    """

    df: pd.DataFrame
    reviews_scraped: pd.DataFrame

    df_product_title_encoded: BertInputBatch
    """shape: (N, MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""

    df_review_title_encoded: BertInputBatch
    """shape: (N, MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""

    df_review_text_encoded: BertInputBatch
    """shape: (N, MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""

    review_feature_vector: torch.Tensor
    """shape: (N, 4)    Rating [1-5; 1] + Verified Purchase [0-1; 1] + SA Valence + Subjectivity [0.0-1.0; 2]"""

    product_feature_vector: torch.Tensor
    """shape: (N, 37)    Ratings (1-5) Ratio [0.0-1.0]; 5] + Rating Avg [f64; 1] + Rating Count [u64; 1] + Category [u64; 30]"""

    label_vector: torch.Tensor
    """shape: (N, 1) """

    scraped_review_feature_vector: torch.Tensor
    """shape: (M, 4)    Rating [1-5; 1] + Verified Purchase [0-1; 1] + SA Valence + Subjectivity [0.0-1.0; 2]"""

    scraped_review_helpful: torch.Tensor
    """shape: (M, 1)    Helpfulness [u64; 1]"""

    scraped_review_title_encoded: BertInputBatch
    """shape: (M, MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""

    scraped_review_text_encoded: BertInputBatch
    """shape: (M, MAX_SEQUENCE_LENGTH_BERT_TOKENIZER)"""

    # product_id_indexes_map: dict[str, list[int]]
    # """For each product, maps the product id to the int indices for all scraped context reviews for this product"""

    def __init__(self, df, reviews_scraped):

        self.df = df
        self.reviews_scraped = reviews_scraped
        M = reviews_scraped.__len__()
        N = df.__len__()

        # use bert tokenizer to tokenize all strings of the
        print(
            f"creating ReviewsDataSet (real/fake reviews: N={len(df)}, context reviews: M={len(reviews_scraped)}"
        )
        print(f"    bert tokeinzer working...")

        def bert_input_batch_tokenize(
            #list_of_strings: list[str],
            list_of_strings: list,
        ) -> BertInputBatch:
            list_of_strings = [str(e) for e in list_of_strings]
            return BertInputBatch.from_batch_encoding(
                BERT_TOKENIZER.batch_encode_plus(
                    list_of_strings,
                    max_length=BERT_TOKENIZER_LENGTH,
                    pad_to_max_length=True,
                    truncation=True,
                    return_token_type_ids=False,
                )
            )

        print(f"    creating df_product_title_encoded...")
        self.df_product_title_encoded = bert_input_batch_tokenize(
            df["product_title"].tolist()
        )

        print(f"    creating df_review_title_encoded...")
        self.df_review_title_encoded = bert_input_batch_tokenize(
            df["review_title"].tolist()
        )

        print(f"    creating df_review_text_encoded...")
        self.df_review_text_encoded = bert_input_batch_tokenize(
            df["review_text"].tolist()
        )

        print(f"    creating scraped_review_title_encoded...")
        self.scraped_review_title_encoded = bert_input_batch_tokenize(
            reviews_scraped["review_title"].tolist()
        )

        print(f"    creating scraped_review_text_encoded...")
        self.scraped_review_text_encoded = bert_input_batch_tokenize(
            reviews_scraped["review_text"].tolist()
        )

        review_feature_cols = [
            "rating",
            "verified",
            "text_sentiment",
            "text_subjectivity",
        ]
        self.review_feature_vector = torch.tensor(df[review_feature_cols].to_numpy()).float().to(DEVICE)
        self.label_vector = torch.tensor(df[["label"]].to_numpy()).float().to(DEVICE)
        self.scraped_review_feature_vector = torch.tensor(
            reviews_scraped[review_feature_cols].to_numpy()
        ).float().to(DEVICE)
        self.scraped_review_helpful = torch.reshape(
            torch.tensor(reviews_scraped["helpful"].to_numpy()).float(), (M, 1)
        ).to(DEVICE)

        product_feature_cols = [
            "rating_count",
            "rating_avg",
            "rating1",
            "rating2",
            "rating3",
            "rating4",
            "rating5",
        ] + [f"cat_{i}" for i in range(0, 30)]
        self.product_feature_vector = torch.tensor(df[product_feature_cols].to_numpy()).float().to(DEVICE)

    #def __getitem__(self, index) -> tuple[torch.Tensor, torch.Tensor]:
    def __getitem__(self, index):
        product_id = self.df.loc[index, "product_id"]

        # get indexes in scraped data where product id is the same
        indices = torch.tensor(
            reviews_scraped.index[
                reviews_scraped["product_id"] == product_id
            ].to_numpy()
        ).int().to(DEVICE)

        def slice(tensor: torch.Tensor) -> torch.Tensor:
            """selects only the subset where index indicates product id is the same"""
            return torch.index_select(tensor, 0, indices)

        def zero_pad_ravel(tensor: torch.Tensor, X) -> torch.Tensor:
            """takes in a tensor of shape (N,D) appends zero elements or removes elements from the end to create an (X,D) shaped tensor and then reshapes that into a (X*D) shaped tensor"""
            (N, D) = tensor.shape
            return torch.reshape(
                torch.nn.functional.pad(
                    input=tensor[0:X, :],
                    pad=(0, 0, 0, max(X - N, 0)),
                    mode="constant",
                    value=0,
                ),
                (-1,),
            )

        # slice the tensors of relevant scraped reviews out of the total reviews:
        scraped_helpful = slice(self.scraped_review_helpful)
        _N = scraped_helpful.shape[0]  # is N
        assert scraped_helpful.shape == (_N, 1)

        scraped_review_features = slice(self.scraped_review_feature_vector)
        assert scraped_review_features.shape == (_N, 4)

        scraped_review_texts_att = slice(
            self.scraped_review_text_encoded.attention_mask
        )

        assert scraped_review_texts_att.shape == (_N, BERT_TOKENIZER_LENGTH)

        scraped_review_texts_ids = slice(self.scraped_review_text_encoded.input_ids)
        assert scraped_review_texts_ids.shape == (_N, BERT_TOKENIZER_LENGTH)

        scraped_review_titles_att = slice(
            self.scraped_review_title_encoded.attention_mask
        )
        assert scraped_review_titles_att.shape == (_N, BERT_TOKENIZER_LENGTH)

        scraped_review_titles_ids = slice(self.scraped_review_title_encoded.input_ids)
        assert scraped_review_titles_ids.shape == (_N, BERT_TOKENIZER_LENGTH)

        # combine all review data into a (X,) shaped tensor:
        scraped_block = torch.cat(
            (
                scraped_helpful,
                scraped_review_features,
                scraped_review_texts_att,
                scraped_review_texts_ids,
                scraped_review_titles_att,
                scraped_review_titles_ids,
            ),
            dim=1,
        )
        scraped_block_flat = zero_pad_ravel(scraped_block, SCRAPED_REVIEW_LIMIT)
        assert scraped_block_flat.shape == (
            SCRAPED_REVIEW_LIMIT * (1 + 4 + 4 * BERT_TOKENIZER_LENGTH),
        )

        product_title_bert_input_att = self.df_product_title_encoded[index][
            "attention_mask"
        ][0, :]
        assert product_title_bert_input_att.shape == (BERT_TOKENIZER_LENGTH,)
        product_title_bert_input_ids = self.df_product_title_encoded[index][
            "input_ids"
        ][0, :]
        assert product_title_bert_input_ids.shape == (BERT_TOKENIZER_LENGTH,)
        review_title_bert_input_att = self.df_review_title_encoded[index][
            "attention_mask"
        ][0, :]
        assert review_title_bert_input_att.shape == (BERT_TOKENIZER_LENGTH,)
        review_title_bert_input_ids = self.df_review_title_encoded[index]["input_ids"][
            0, :
        ]
        assert review_title_bert_input_ids.shape == (BERT_TOKENIZER_LENGTH,)
        review_text_bert_input_att = self.df_review_text_encoded[index][
            "attention_mask"
        ][0, :]
        assert review_text_bert_input_att.shape == (BERT_TOKENIZER_LENGTH,)
        review_text_bert_input_ids = self.df_review_text_encoded[index]["input_ids"][
            0, :
        ]
        assert review_text_bert_input_ids.shape == (BERT_TOKENIZER_LENGTH,)

        product_features = self.product_feature_vector[index]  # shape: (37)
        assert product_features.shape == (37,)

        review_features = self.review_feature_vector[index]  # shape: (4)
        assert review_features.shape == (4,)

        catted = torch.cat(
            (
                product_features,
                review_features,
                product_title_bert_input_att,
                product_title_bert_input_ids,
                review_title_bert_input_att,
                review_title_bert_input_ids,
                review_text_bert_input_att,
                review_text_bert_input_ids,
                scraped_block_flat,
            ),
            dim=0,
        )
        assert catted.shape == (PACKED_DATASET_ROW_SIZE,)
        ##############################################################
        ## Data layout in the catted tensor row, assuming BERT_TOKENIZER_LENGTH = 256 and SCRAPED_REVIEW_LIMIT = 10
        ## | 37 | 4 | 256+256 | 256+256 | 256+256 | 10 * ( | 1 | 4 | 256+256 | 256+256 |)

        label = self.label_vector[index]
        return (catted, label)

    def __len__(self) -> int:
        return self.df.__len__()

#### unpacking functions

In [21]:
def unpack_dataset_items(packed_batch_tensor: torch.Tensor):
    """
    expects packed_batch_tensor to be of shape
    """
    assert packed_batch_tensor.shape[1] == PACKED_DATASET_ROW_SIZE
    (B, T) = packed_batch_tensor.shape

    c: int = 0

    def next(n: int) -> torch.Tensor:
        nonlocal c
        slice = packed_batch_tensor[:, c : c + n]
        c += n
        return slice

    product_features = next(37)
    assert product_features.shape == (B, 37)

    review_features = next(4)
    assert review_features.shape == (B, 4)

    product_title_bert_input_att = next(BERT_TOKENIZER_LENGTH)
    product_title_bert_input_ids = next(BERT_TOKENIZER_LENGTH)
    review_title_bert_input_att = next(BERT_TOKENIZER_LENGTH)
    review_title_bert_input_ids = next(BERT_TOKENIZER_LENGTH)
    review_text_bert_input_att = next(BERT_TOKENIZER_LENGTH)
    review_text_bert_input_ids = next(BERT_TOKENIZER_LENGTH)
    assert (
        product_title_bert_input_att.shape
        == product_title_bert_input_ids.shape
        == review_title_bert_input_att.shape
        == review_title_bert_input_ids.shape
        == review_text_bert_input_att.shape
        == review_text_bert_input_ids.shape
        == (B, BERT_TOKENIZER_LENGTH)
    )

    _scraped_block_flat = next(
        SCRAPED_REVIEW_LIMIT * (1 + 4 + 4 * BERT_TOKENIZER_LENGTH)
    )
    assert _scraped_block_flat.shape == (
        B,
        SCRAPED_REVIEW_LIMIT * (1 + 4 + 4 * BERT_TOKENIZER_LENGTH),
    )

    scraped_block = torch.reshape(
        _scraped_block_flat,
        (B, SCRAPED_REVIEW_LIMIT, (1 + 4 + 4 * BERT_TOKENIZER_LENGTH)),
    )
    assert scraped_block.shape == (
        B,
        SCRAPED_REVIEW_LIMIT,
        (1 + 4 + 4 * BERT_TOKENIZER_LENGTH),
    )

    scraped_helpful = scraped_block[:, :, 0:1]
    assert scraped_helpful.shape == (B, SCRAPED_REVIEW_LIMIT, 1)

    scraped_review_features = scraped_block[:, :, 1:5]
    assert scraped_review_features.shape == (B, SCRAPED_REVIEW_LIMIT, 4)

    scraped_review_title_bert_input_att = scraped_block[
        :, :, 5 : 5 + BERT_TOKENIZER_LENGTH
    ]
    scraped_review_title_bert_input_ids = scraped_block[
        :, :, 5 + BERT_TOKENIZER_LENGTH : 5 + 2 * BERT_TOKENIZER_LENGTH
    ]
    scraped_review_text_bert_input_att = scraped_block[
        :, :, 5 + 2 * BERT_TOKENIZER_LENGTH : 5 + 3 * BERT_TOKENIZER_LENGTH
    ]
    scraped_review_text_bert_input_ids = scraped_block[
        :, :, 5 + 3 * BERT_TOKENIZER_LENGTH : 5 + 4 * BERT_TOKENIZER_LENGTH
    ]

    return (
        product_features,
        review_features,
        product_title_bert_input_att,
        product_title_bert_input_ids,
        review_title_bert_input_att,
        review_title_bert_input_ids,
        review_text_bert_input_att,
        review_text_bert_input_ids,
        (
            scraped_helpful,
            scraped_review_features,
            scraped_review_title_bert_input_att,
            scraped_review_title_bert_input_ids,
            scraped_review_text_bert_input_att,
            scraped_review_text_bert_input_ids,
        ),
    )

### Model

In [22]:
from collections import OrderedDict
import torch.nn as nn
#def create_mlp(layer_sizes: list[int]) -> nn.Sequential:
def create_mlp(layer_sizes: list):
    """
    creates an MLP with the given layer_sizes. The first element is the input size, the last one the output size
    args:
        layer_sizes: [input_dim, h1_dim, h2_dim, ...., out_dim]"""
    assert layer_sizes.__len__() >= 2
        
    layers = []
    for i in range(1, layer_sizes.__len__()):
        layers.append((f"hidden_layer_{i}",nn.Linear(layer_sizes[i-1], layer_sizes[i]) ))
        layers.append((f"activation_{i}",  nn.ReLU()))
    return nn.Sequential(OrderedDict(layers))

In [23]:
class ReviewEncodingModel(torch.nn.Module):

    def __init__(self, freeze: bool = True, outdim=500) -> None:
        super(ReviewEncodingModel, self).__init__()
        self.bert = BertModel(BERT_CONFIG)
        if freeze:
            for param in self.bert.parameters():
                param.requires_grad = False
        self.out_mlp = create_mlp([BERT_EMBEDDING_SIZE*2+4, outdim])
    
    def forward(self, review_features: torch.Tensor, 
               review_title_bert_input_att: torch.Tensor,
               review_title_bert_input_ids: torch.Tensor,
               review_text_bert_input_att: torch.Tensor,
               review_text_bert_input_ids: torch.Tensor, ) -> torch.Tensor:
        """
        args:
            review_features: shape: (BATCH_SIZE, 5)
            review_title_bert_input_att: shape: (BATCH_SIZE, BERT_TOKENIZER_LENGTH)
            review_title_bert_input_ids: shape: (BATCH_SIZE, BERT_TOKENIZER_LENGTH)
            review_text_bert_input_att: shape: (BATCH_SIZE, BERT_TOKENIZER_LENGTH)
            review_text_bert_input_ids: shape: (BATCH_SIZE, BERT_TOKENIZER_LENGTH)
        returns:
            torch.Tensor of shape (BATCH_SIZE, OUTDIM)
        """

        # max over all tokens in every dimension of the 768 dimensional embedding
      
        bert_title_embedding = self.bert(attention_mask=review_title_bert_input_att.int(), input_ids=review_title_bert_input_ids.int()).last_hidden_state[:,0,:]
        assert bert_title_embedding.shape == (BATCH_SIZE, BERT_EMBEDDING_SIZE)

        # max over all tokens in every dimension of the 768 dimensional embedding
        bert_text_embedding = self.bert(attention_mask=review_text_bert_input_att.int(), input_ids=review_text_bert_input_ids.int()).last_hidden_state[:,0,:]
        assert bert_title_embedding.shape == (BATCH_SIZE, BERT_EMBEDDING_SIZE)

        # concat all on top of each other:
        catted = torch.cat([bert_title_embedding, bert_text_embedding, review_features], dim=1)

        # apply linear layer and relu:
        return self.out_mlp(catted)

        
    def freeze_bert(self, freezed: bool) -> None:
        for param in self.bert.parameters():
                param.requires_grad =  not freezed


# Fake Detection Model

In [24]:
REVIEW_ENCODING_MODEL_OUTDIM = 149
# REVIEW_ENCODING_MODEL_OUTDIM + 1 has to be divisible by 3 for this to work, bc. we use 3 attention heads

class FakeDetectionModel(torch.nn.Module):
    review_encoding_model: ReviewEncodingModel
    scraped_reviews_transformer: torch.nn.TransformerEncoderLayer
    outmlp: nn.Sequential

    def __init__(self):
        super(FakeDetectionModel, self).__init__()
        self.review_encoding_model: ReviewEncodingModel = ReviewEncodingModel(
            freeze=True, outdim=REVIEW_ENCODING_MODEL_OUTDIM
        )
        self.scraped_reviews_transformer = torch.nn.TransformerEncoderLayer(
            d_model=REVIEW_ENCODING_MODEL_OUTDIM+1, nhead=15, batch_first=True
        )

        LAST_FEATURES_DIM = 37 + (REVIEW_ENCODING_MODEL_OUTDIM + 1) + REVIEW_ENCODING_MODEL_OUTDIM +  BERT_EMBEDDING_SIZE
        # LAST_FEATURES_DIM: product features + transformer output dimension + review encoded + bert embedding from product title
        self.outmlp = create_mlp([LAST_FEATURES_DIM, 30, 20, 1])

    def forward(self, packed_dataset_rows: torch.Tensor)-> torch.Tensor :
        """ "
        args:
            packed_dataset_row: torch.Tensor with shape (BATCH_SIZE, PACKED_DATASET_ROW_SIZE)
        returns:
            torch.Tensor with shape: (BATCH_SIZE,1)
        """
        #print("packed_dataset_rows.shape",packed_dataset_rows.shape )
        #print("BATCH_SIZE, PACKED_DATASET_ROW_SIZE", BATCH_SIZE, PACKED_DATASET_ROW_SIZE)


        assert packed_dataset_rows.shape == (BATCH_SIZE, PACKED_DATASET_ROW_SIZE)
        (
            product_features,
            review_features,
            product_title_bert_input_att,
            product_title_bert_input_ids,
            review_title_bert_input_att,
            review_title_bert_input_ids,
            review_text_bert_input_att,
            review_text_bert_input_ids,
            (
                scraped_helpful,
                scraped_review_features,
                scraped_review_title_bert_input_att,
                scraped_review_title_bert_input_ids,
                scraped_review_text_bert_input_att,
                scraped_review_text_bert_input_ids,
            ),
        ) = unpack_dataset_items(packed_dataset_rows)

        ### CREATE REVIEW ENCODING
        review_encoding = self.review_encoding_model(review_features, review_title_bert_input_att, review_title_bert_input_ids, review_text_bert_input_att, review_text_bert_input_ids)  # type: ignore
        
        assert review_encoding.shape == (BATCH_SIZE, REVIEW_ENCODING_MODEL_OUTDIM)

        ### CREATE REVIEW ENCODINGS FOR ALL SCRAPED REVIEWS
        ### THEN COMBINE THEM AND THEIR HELPFULNESS VIA THE TRANSFORMER

        # transform (BATCH_SIZE, SCRAPED_REVIEW_LIMIT, 5 + 4 * BERT_TOKENIZER_LENGTH) into (BATCH_SIZE, SCRAPED_REVIEW_LIMIT, REVIEW_ENCODING_MODEL_OUTDIM)
        transformer_input = torch.zeros(
            (BATCH_SIZE, SCRAPED_REVIEW_LIMIT, 1 + REVIEW_ENCODING_MODEL_OUTDIM)
        ).to(DEVICE)
        for i in range(SCRAPED_REVIEW_LIMIT):
            transformer_input[:, i, :] = torch.cat(
            [self.review_encoding_model(
                scraped_review_features[:, i, :],
                scraped_review_title_bert_input_att[:, i, :],
                scraped_review_title_bert_input_ids[:, i, :],
                scraped_review_text_bert_input_att[:, i, :],
                scraped_review_text_bert_input_ids[:, i, :],
            ), scraped_helpful[:,i,:]], dim=1 ) # type: ignore
        # REVIEW_ENCODING_MODEL_OUTDIM + 1 has to be divisible by 3 for this to work, bc. we use 3 attention heads

        transformer_output = self.scraped_reviews_transformer(transformer_input)[:,-1,:] # type: ignore
        assert transformer_output.shape == (BATCH_SIZE, 1+ REVIEW_ENCODING_MODEL_OUTDIM)

        ### CREATE BERT EMBEDDING FOR PRODUCT TITLE
        bert_product_title_embedding = self.review_encoding_model.bert(attention_mask=product_title_bert_input_att.int(), input_ids=product_title_bert_input_ids.int()).last_hidden_state[:,0,:] # type: ignore
        assert bert_product_title_embedding.shape == (BATCH_SIZE, BERT_EMBEDDING_SIZE)
        # linear layer connection all

        ### COMBINE PRODUCT INFORMATION (features + title), CONTEXT (other review's encodings and their helpfulness) and REVIEW ENCODING into a single scalar: the real vs. fake prediction
        catted = torch.cat([product_features, bert_product_title_embedding, review_encoding, transformer_output],dim=1)
        return torch.sigmoid(self.outmlp(catted)) # type: ignore

### Running

## Create Datasets

In [25]:
dataset = ReviewsDataSet(df, reviews_scraped)
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE)

creating ReviewsDataSet (real/fake reviews: N=18107, context reviews: M=99889
    bert tokeinzer working...
    creating df_product_title_encoded...


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


    creating df_review_title_encoded...
    creating df_review_text_encoded...
    creating scraped_review_title_encoded...
    creating scraped_review_text_encoded...


In [26]:
# import torch.utils.data as data
# train_set_size = int(len(dataset) * 0.8)
# val_set_size = len(dataset) - train_set_size
# train_dataset, val_dataset = data.random_split(dataset, [train_set_size, val_set_size])
# train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,drop_last=True)
# val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,drop_last=True)

In [27]:
import torch.utils.data as data
train_set_size = int(len(dataset) * 0.8)
val_set_size = len(dataset) - train_set_size
train_dataset, val_dataset = data.random_split(dataset, [train_set_size, val_set_size])
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,drop_last=True)

new_val_set_size = int(len(val_dataset) * 0.5)
test_set_size = val_set_size - new_val_set_size
val_dataset, test_dataset = data.random_split(val_dataset, [new_val_set_size, test_set_size])
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,drop_last=True)
test_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,drop_last=True)

In [28]:
for (input, labels) in val_dataloader:
    ( product_features,
        review_features,
        product_title_bert_input_att,
        product_title_bert_input_ids,
        review_title_bert_input_ids,
        review_title_bert_input_ids,
        review_text_bert_input_att,
        review_text_bert_input_ids,
        scraped_block ) = unpack_dataset_items(input)
    # this is just a test if we can get items from the dataset
    break

In [29]:
model = FakeDetectionModel().to(DEVICE)


# TRAIN

In [30]:
from sklearn.metrics import accuracy_score
def metrics(y, y_hat):
  y_hat = np.nan_to_num(y_hat, nan = 0.0001)
  
  return accuracy_score(y, y_hat.round()) * 100

In [31]:
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
def metrics(y, y_hat, typeMetric = 'PRC' ):
    
    precision, recall, thresholds = precision_recall_curve(y, y_hat)

    if typeMetric == 'f1':
      fscore = (2 * precision * recall) / (precision + recall)
      ix = np.argmax(fscore)
      return  fscore[ix]
    elif typeMetric == 'recall':
      ix = np.argmax(recall)
      return recall[ix]
    elif typeMetric == 'precision':
      ix = np.argmax(precision)
      return precision[ix]
    elif typeMetric == 'AUROC':
      return roc_auc_score(y, y_hat)
    elif typeMetric == 'PRC':
      return auc(recall, precision)
    elif typeMetric == 'acc':
      return accuracy_score(y, y_hat.round()) * 100

In [32]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr = 1e-3)
#optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)



/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [33]:
loss_f = nn.BCELoss()
#loss_f = nn.BCEWithLogitsLoss()

In [ ]:
from tqdm.notebook import tqdm_notebook
epochs = 10

process_factor = 1 # show every n times


pbarEpoch = tqdm_notebook(total = epochs)
pbarEpoch.set_description('Epoch: ')


for epoch in range(epochs):
  y_hat = []
  y = []
  flag = True

  loss_train = 0.0
  acc_train  = 0.0

  index = 0


  model.train()
  for (input, labels) in train_dataloader:
    
    o = model(input)

   

    # with torch.no_grad():
    #   print(model.state_dict())
    # o = torch.nan_to_num(o, 0.001)
    # labels = torch.nan_to_num(labels, 0.001)
    loss = loss_f(o, labels)

    

    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

    
    
    predicted = o.cpu().detach().numpy()
    ground_truth = labels.cpu().detach().numpy()
    if flag:
          y_hat = predicted
          y = ground_truth
          flag = False

    y_hat = np.append(y_hat, predicted)
    y = np.append(y, ground_truth)

    acc_train = metrics(y, y_hat)
    loss_train = loss.item()

    if (index) % process_factor == 0:
      
      text = f'Training Stage ==> Epoch: {epoch} / {epochs - 1} | Step: {index} / {len(train_dataloader)} | Training loss: {loss.item():.5f} |  Training acc: {acc_train:.5f}'
      print(text)

    pbarEpoch.set_postfix({'Epoch':epoch,
            'Training loss': loss_train, 
            'Training Acc': acc_train, 
            'Step': index,
            'from': len(train_dataloader)
            })
    
    
    index+=1
  
  text = f'Training Stage ==> Epoch: {epoch} / {epochs - 1} | Training loss: {loss_train:.5f} |  Training Accuracy: {acc_train:.5f}'
  print(text)

# VALIDATE    

    

  index = 0
  y_hat = []
  y = []
  flag = True

  loss_val = 0.0
  acc_val  = 0.
  index = 0
  with torch.no_grad():
    model.eval()
    for (input, labels) in val_dataloader:
      o = model(input)
      # o = torch.nan_to_num(o, 0.001)
      loss = loss_f(o, labels)
      
      
      predicted = o.cpu().detach().numpy()
      ground_truth = labels.cpu().detach().numpy()
      if flag:
          y_hat = predicted
          y = ground_truth
          flag = False

      y_hat = np.append(y_hat, predicted)
      y = np.append(y, ground_truth)

      acc_val = metrics(y, y_hat)
      loss_val = loss.item()

      pbarEpoch.set_postfix({'Epoch':epoch,
            'Validation loss': loss_val, 
            'Validation Acc': acc_val, 
            'Step': index,
            'from': len(val_dataloader)
            })
      
      index +=1 

    text = f'Validation Stage ==> Epoch: {epoch} / {epochs - 1} | Validation loss: {loss_val:.5f} |  Validation Accuracy: {acc_val:.5f}'
    print(text)

  pbarEpoch.update(1)
  pbarEpoch.close()

  0%|          | 0/10 [00:00<?, ?it/s]

Training Stage ==> Epoch: 0 / 9 | Step: 0 / 56 | Training loss: 1.43231 |  Training acc: 0.55329
Training Stage ==> Epoch: 0 / 9 | Step: 1 / 56 | Training loss: 0.69896 |  Training acc: 0.54749
Training Stage ==> Epoch: 0 / 9 | Step: 2 / 56 | Training loss: 0.69315 |  Training acc: 0.53941
Training Stage ==> Epoch: 0 / 9 | Step: 3 / 56 | Training loss: 0.69315 |  Training acc: 0.53424
Training Stage ==> Epoch: 0 / 9 | Step: 4 / 56 | Training loss: 0.69315 |  Training acc: 0.53179
Training Stage ==> Epoch: 0 / 9 | Step: 5 / 56 | Training loss: 0.69315 |  Training acc: 0.52933
Training Stage ==> Epoch: 0 / 9 | Step: 6 / 56 | Training loss: 0.69315 |  Training acc: 0.52512
Training Stage ==> Epoch: 0 / 9 | Step: 7 / 56 | Training loss: 0.69315 |  Training acc: 0.52342
Training Stage ==> Epoch: 0 / 9 | Step: 8 / 56 | Training loss: 0.69315 |  Training acc: 0.52250
Training Stage ==> Epoch: 0 / 9 | Step: 9 / 56 | Training loss: 0.69315 |  Training acc: 0.52144
Training Stage ==> Epoch: 0 / 

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
torch.cuda.empty_cache()

# EVAL

In [ ]:
model.eval()
all_preds = []
all_labels = []
for (input, labels) in val_dataloader:
    o = model(input)
    #print(o)
    all_preds.append(o.cpu().detach().numpy())
    all_labels.append(labels.cpu().detach().numpy())

In [ ]:
all_labels = np.asarray(all_labels)
all_preds_np = np.asarray(all_preds)
np.save("all_preds.npy",all_preds)

In [ ]:
!pip install binclass-tools

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

def show_conf_matrix(target, preds, set1 = "", model="", run=None):
    preds = np.argmax(preds, axis = 1)
    confmat_DNN = confusion_matrix(target, preds)
    print("CM - " +set1 +  " Set")
    print("-----------")
    print(confmat_DNN)
    plt.figure(figsize=(4,4))
    fig = sns.heatmap(confmat_DNN, annot=True,  linewidths=.5, square = True, cmap = 'Blues')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("CM - " +set1 +  " Set" , size = 15)

    if run:
        run["confusion-matrix"] = fig

In [ ]:
import bctools as bc
import numpy.ma as ma
o = all_preds
target = all_labels[0]
#o_np = o.cpu().detach().numpy()
preds = np.where(np.isnan(o), ma.array(o, mask=np.isnan(o)).mean(axis=0), o)    
#target = labels.cpu().detach().numpy()



auROC_test = bc.curve_ROC_plot(true_y= target, predicted_proba = preds, title = "AUROC - Test" )  
auPRC_test = bc.curve_PR_plot(true_y= target, predicted_proba = preds,  beta = 1, title = "AUPRC - Test" )
show_conf_matrix(target, preds, set1 = "Test", model="", run=None)